# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry==0.6.10.dev3
```

## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure, BackendWrapper
from qiskit import QuantumCircuit

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `BackendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `BackendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = BackendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [3]:
print("| Does we have GPU backend to access:", backend.is_aer_gpu)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [7]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [8]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'4b8087d4-3e7f-484c-b57a-b7ff0bfc415d'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [9]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    4b8087d4-3e7f-484c-b57a-b7ff0bfc415d: ...
} with 1 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [10]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=4b8087d4-3e7f-484c-b57a-b7ff0bfc415d, Arguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='4b8087d4-3e7f-484c-b57a-b7ff0bfc415d', wave_key='TrivialParamagnet', shots=1024, backend=GeneralAerSimulator('aer_simulator'), run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-01-13 05:35:18', 'run': '2024-01-13 05:35:18'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [11]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=4b8087d4-3e7f-484c-b57a-b7ff0bfc415d
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ 4b8087d4-3e7f-484c-b57a-b7ff0bfc415d   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- aer_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '2

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

In [12]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(0, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=4
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [13]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.0444787216186524, entropy=-0.06278310176074925, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-01-13 05:35:19', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [14]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.0444787216186524, entropy=-0.06278310176074925, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-01-13 05:35:19', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [15]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [16]:
main

{'purity': 1.0444787216186524,
 'entropy': -0.06278310176074925,
 'puritySD': 0.9302319076038817,
 'entropySD': 1.2848906657447356,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.3166787910461426,
 'entropyAllSys': -0.39690343740524925,
 'puritySDAllSys': 1.6631627386840024,
 'entropySDAllSys': 1.822340157377556,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.14800725984555,
 'mitigatedPurity': 0.8075969887425068,
 'mitigatedEntropy': 0.3082925635324282,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000742268,
 'takingTimeAllSys': 0.003371726,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-01-13 05:35:19',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [17]:
side_prodct

{'purityCells': {89: 2.6989498138427734,
  18: 0.5346031188964844,
  86: 0.3613567352294922,
  94: 2.488922119140625,
  96: 0.3631706237792969,
  8: 1.3395576477050781,
  83: 0.37652587890625,
  43: 0.5356273651123047,
  68: 6.351995468139648,
  80: 1.224081039428711,
  36: 1.673971176147461,
  59: 0.4213752746582031,
  22: 0.7196540832519531,
  95: 1.2671852111816406,
  65: 0.9156684875488281,
  25: 0.788330078125,
  37: 0.4486865997314453,
  39: 1.197519302368164,
  88: 1.0248336791992188,
  26: 1.0326385498046875,
  69: 1.6838417053222656,
  79: 2.258237838745117,
  30: 0.5025596618652344,
  44: 0.3161640167236328,
  49: 0.5880584716796875,
  87: 0.8497734069824219,
  52: 1.576416015625,
  67: 2.1063919067382812,
  14: 0.8012104034423828,
  92: 0.6277523040771484,
  74: 2.039976119995117,
  72: 0.2652149200439453,
  24: 0.7666149139404297,
  55: 0.1875782012939453,
  61: 0.8987998962402344,
  99: 0.42880821228027344,
  4: 0.4293060302734375,
  47: 2.9651050567626953,
  81: 0.3989162

In [18]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-01-13 05:35:19
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (0, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.0444787216186524
   - entropy ---------------------- -0.06278310176074925
   - puritySD --------------------- 0.9302319076038817
   - entropySD -------------------- 1.2848906657447356
   - purityCells ------------------ {89: 2.6989498138427734, 18: 0.5346031188964844, 86: 0.3613567352294922, 94: 2.488922119140625, 96: 0.3631706237792969, 8: 1.3395576477050781, 83: 0.37652587890625, 43: 0.5356273651123047, 68: 6.351995468139648, 80: 1.224081039428711, 36: 1.673971176147461, 59: 0.4213752746582031, 22: 0.719

---

## Post-Process Availablities and Version Info

In [19]:
from qurry import BackendAvailabilities as root_availability, __version__
from qurry.process.randomized_measure.entangled_entropy import (
    BackendAvailabilities as entangled_availability,
)
from qurry.process.randomized_measure.purity_cell import (
    BackendAvailabilities as purity_cell_availability,
)
from qurry.process.randomized_measure.wavefunction_overlap import (
    BackendAvailabilities as overlap_availability,
)
from qurry.process.randomized_measure.echo_cell import (
    BackendAvailabilities as echo_cell_availability,
)
from qurry.process.utils.randomized import (
    BackendAvailabilities as randomized_availability,
)
from qurry.process.utils.construct import (
    BackendAvailabilities as construct_availability,
)

availability_dict = [
    ("root", root_availability),
    ("entangled", entangled_availability),
    ("purity", purity_cell_availability),
    ("overlap", overlap_availability),
    ("echo", echo_cell_availability),
    ("randomized", randomized_availability),
    ("construct", construct_availability),
]

print("| Qurry version:", __version__)
for name, availability in availability_dict:
    print(f"| - The availability of {name}")
    for key, value in availability.items():
        print(f"|   - {key}: {value}")

| Qurry version: (0, 6, 11, 'dev1')
| - The availability of root
|   - Python: True
|   - Rust: True
| - The availability of entangled
|   - Cython: True
|   - Rust: True
|   - Python: True
| - The availability of purity
|   - Cython: True
|   - Rust: True
|   - Python: True
| - The availability of overlap
|   - Cython: True
|   - Python: True
| - The availability of echo
|   - Cython: True
|   - Python: True
| - The availability of randomized
|   - Cython: True
|   - Rust: True
|   - Python: True
| - The availability of construct
|   - Rust: True
|   - Python: True
